In [9]:
pip install pyspark

Note: you may need to restart the kernel to use updated packages.


In [15]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import size,stddev 
from pyspark.sql.types import *
from pyspark.sql.functions import mean, col
import pyspark.sql.functions as func




# spark = SparkSession.builder.master("local")\
#         .config("spark.port.maxRetries", 100)\
#         .config("spark.executor.instances", "6")\
#         .config("spark.executor.cores", "4")\
#         .config("spark.executor.memory", "8G")\
#         .config("spark.driver.memory", "2G")\
#         .config("spark.dynamicAllocation.enabled", "false")\
#         .config("spark.yarn.queue", "Low")\
#         .config("spark.port.maxRetries", 100)\
#         .appName("New_Data_Reader_V0")\
#         .getOrCreate()

spark = SparkSession.builder.master("local").getOrCreate()

In [17]:
%%capture cap --no-stderr

years=['2010','2011','2012','2013','2014','2015','2016','2017','2018','2019','2020','2021','2022']
#Lopp over the years and process the data
for year in years:
    # Read the CSV files for the current year
    path="/Users/hasinirasagna/Downloads/data/"+year
    df = spark.read.option("header", "true").option("inferSchema", "true").option("delimiter",",").csv(path+"/*.csv")
    df.createOrReplaceTempView("temp_view")
    # Find the hottest day for the year
    temperature1 = spark.sql("SELECT STATION,NAME,MAX, DATE FROM temp_view WHERE MAX = (SELECT MAX(MAX) FROM temp_view );")
    print("Hottest day for "+ year)
    temperature1.show()
    
    
# Find the coldest day of January across all years
path="/Users/hasinirasagna/Downloads/data/"
df1 = spark.read.option("header", "true").option("inferSchema", "true").option("delimiter",",").csv(path + "/*/*.csv")  
df1.createOrReplaceTempView("COLD")
coldest=spark.sql("SELECT STATION,NAME,DATE,MIN FROM cold WHERE MIN = (SELECT MIN(MIN) FROM cold WHERE DATE LIKE '%-01-%') AND DATE LIKE '%-01-%' ORDER BY DATE")
print("Cold Day of January")
coldest.show()

# Find the maximum and minimum precipitation for year 2015
temperature2 = spark.sql("SELECT STATION,NAME,PRCP, DATE FROM COLD WHERE PRCP = (SELECT MAX(PRCP) FROM COLD where PRCP!=99.99  AND YEAR(DATE) = 2015) and YEAR(DATE) = 2015 limit 1")
temperature3 = spark.sql("SELECT STATION,NAME,PRCP, DATE FROM COLD WHERE PRCP = (SELECT MIN(PRCP) FROM COLD where PRCP!=0.0) and YEAR(DATE) = 2015 limit 1")
print(" 3. Max Precipitation for year 2015")
temperature2.show()
print("3. Min Precipitation for year 2015")
temperature3.show()

# Calculate the percentage of no gust values for year 2019
df1 = spark.read.option("header", "true").option("inferSchema", "true").option("delimiter",",").csv(path + "/*/*.csv")  
df1.createOrReplaceTempView("GUSTY")
No_gust=spark.sql("Select count(GUST) from GUSTY WHERE YEAR(DATE) = 2019 AND GUST==999.9").collect()[0]
No_gust=No_gust["count(GUST)"]
TotalCount=spark.sql("Select count(GUST) from GUSTY WHERE YEAR(DATE) = 2019").collect()[0]
TotalCount=TotalCount["count(GUST)"]
print('Percentage of Null Gust values is ',str((No_gust/TotalCount)*100))

# Calculate the mean, median, mode, and standard deviation of temperature for year 2020
df1.createOrReplaceTempView("NEW")
temperature6 = spark.sql("SELECT TEMP,MONTH(DATE) FROM NEW where YEAR(DATE) = 2020")
print("Mean of TEMP in 2020")
temperature6.groupBy("MONTH(DATE)").agg(mean("TEMP")).orderBy("MONTH(DATE)").show(truncate=False)
print("Median of TEMP in 2020")
temperature6.groupBy("MONTH(DATE)").agg(func.percentile_approx("TEMP", 0.5).alias("median")).orderBy("MONTH(DATE)").show()
print("Mode of TEMP in 2020")
mode=spark.sql("with tv1 as (Select month(date) as MONTH, temp,count(temp) over (partition by temp, month(date)) as cnt from NEW where YEAR(DATE) = 2020),  tv2 as (select distinct MONTH, max(cnt) over(partition by MONTH) mx from tv1) select distinct tv2.MONTH , max(tv1.temp) over (partition by tv2.MONTH) as Mode from tv1, tv2 where  tv1.MONTH=tv2.MONTH and tv1.cnt=tv2.mx order by tv2.MONTH") 
mode.show()
print("Standard Deviation of TEMP in 2020")
temperature6.groupBy("MONTH(DATE)").agg(stddev("TEMP")).orderBy("MONTH(DATE)").show(truncate=False)
with open('/Users/hasinirasagna/Downloads/Result.txt', 'w') as f:
    f.write(cap.stdout)